In [54]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim

In [113]:
class contextual_bandit():
    def __init__(self):
        self.state = 0
        #List out our bandits. Currently arms 4, 2, and 1 (respectively) are the most optimal.
        self.bandits = np.array([[0.2,0,-0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self):
        self.state = np.random.randint(0,len(self.bandits)) #Returns a random state for each episode.
        return self.state
        
    def pullArm(self,action):
        #Get a random number.
        bandit = self.bandits[self.state,action]
        result = np.random.randn(1)
        if result > bandit:
            #return a positive reward.
            return 1
        else:
            #return a negative reward.
            return -1

In [104]:
class agent():
    def __init__(self, lr, s_size, a_size):
        self.state_in = tf.placeholder(shape=[1], dtype=tf.int32)
        self.state_in_OH = slim.one_hot_encoding(self.state_in, s_size)
        
        output=slim.fully_connected(self.state_in_OH, a_size, biases_initializer= None,
                            activation_fn=tf.nn.sigmoid,
                             weights_initializer=tf.ones_initializer())

        self.output = tf.reshape(output, [-1])
        self.chosen_action = tf.argmax(self.output, 0)
        
        
        self.reward_holder = tf.placeholder(dtype=tf.float32, shape=[1], name='reward')
        self.action_holder = tf.placeholder(dtype=tf.int32, shape=[1], name='action')
        
        self.responsible_weight = tf.slice(self.output, self.action_holder, [1])
        
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)

In [105]:
tf.reset_default_graph()

cBandits = contextual_bandit()

ag = agent(lr=0.001, s_size=cBandits.num_bandits, a_size=cBandits.num_actions)


total_rewards = np.zeros([cBandits.num_bandits, cBandits.num_actions])

init = tf.global_variables_initializer()
        
e = 0.1 #Set the chance of taking a random action.

    

In [114]:
num_episodes = 2000

with tf.Session() as sess:
    sess.run(init)
    j = 0
    rAll = 0
    while j <= 2000:
        s = cBandits.getBandit()
        j += 1
        if np.random.rand(1) < e:
            action = np.random.randint(cBandits.num_actions)
        else:
            action = sess.run(ag.chosen_action, feed_dict={ag.state_in:[s]})
        
        r = cBandits.pullArm(action)
        
        update = sess.run([ag.update], 
                          feed_dict={ag.reward_holder:[r], 
                                     ag.state_in:[s], 
                                     ag.action_holder:[action]})
        
        total_rewards[s,action] += r
        
        
        

In [115]:
total_rewards

array([[    0.,     7.,     2.,  1379.],
       [    4.,  1569.,   -26.,   -13.],
       [ 1557.,   -45.,   -53.,   -53.]])